In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
model = ChatGroq(model="Gemma2-9b-It")

In [5]:
model.invoke("TEST")

AIMessage(content="Please provide me with the test you'd like me to take! \n\nFor example, do you want me to:\n\n* **Answer a question?**\n* **Summarize a piece of text?**\n* **Translate something?**\n* **Write a story or poem?**\n* **Solve a problem?**\n\n\nLet me know what you have in mind! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 10, 'total_tokens': 94, 'completion_time': 0.152727273, 'prompt_time': 0.001174599, 'queue_time': 0.253671951, 'total_time': 0.153901872}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d5ed9537-3e63-4000-b665-b4b2060c9ca2-0', usage_metadata={'input_tokens': 10, 'output_tokens': 84, 'total_tokens': 94})

In [6]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive","negative"] = Field(description="sentiment of the review")


class DiagnosisSchema(BaseModel):
    issue_type: Literal['UX',"Performance","Bug","Support","Other"] = Field(description="The category of issue mentioned in the review")
    tone: Literal['angry',"frustrated","disappointed","calm"] = Field(description="the emotional tone expressed in the review")
    urgency: Literal['low',"medium","high"] = Field(description="How urgent or critical the issue appears to be")


In [7]:
structured_model = model.with_structured_output(SentimentSchema)
diagnosis_model = model.with_structured_output(DiagnosisSchema)

In [8]:
prompt = f"What is the sentiment of the following review - The software is too bad"

structured_model.invoke(prompt)

SentimentSchema(sentiment='negative')

In [9]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal['positive','negative']
    diagnosis: dict
    response: str


In [25]:
def find_sentiment(state: ReviewState):
    
    prompt = f"For the following review, find out the sentiment \n {state['review']}"
    sentiment = structured_model.invoke(prompt).sentiment
    
    return {'sentiment':sentiment}



def check_sentiment(state: ReviewState) -> Literal["positive_response","run_diagnosis"]:

    if state['sentiment'] == "positive":
        return "positive_response"
    else:
        return "run_diagnosis"


def positive_response(state: ReviewState):

    prompt = f"""Write a warm thank-you message in response to this review:
        \n\n\"{state['review']}\"\n
    Also, kindly ask the user to leave feedback on our website."""
    
    response = model.invoke(prompt).content

    return {'response': response}


def run_diagnosis(state: ReviewState):
    prompt = f"Diagnose this negative review:\n\n{state['review']}\n Return the issue type, tone and urgency"
    response =diagnosis_model.invoke(prompt)

    return {'diagnosis':response.model_dump()}


def negative_response(state: ReviewState):

    diagnosis = state['diagnosis']

    prompt = f"""You are a support assistant.
The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
Write an empathetic, helpful resolution message.
"""
    response = model.invoke(prompt).content

    return {'response': response}
    

In [26]:
graph = StateGraph(ReviewState)

graph.add_node('find_sentiment', find_sentiment)
graph.add_node('positive_response', positive_response)
graph.add_node('run_diagnosis', run_diagnosis)
graph.add_node('negative_response', negative_response)

graph.add_edge(START, 'find_sentiment')

graph.add_conditional_edges('find_sentiment', check_sentiment)

graph.add_edge('positive_response', END)

graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response', END)

workflow = graph.compile()


In [29]:
initial_state = {"review":"This system is trash, nothing works properly and lags alot. Dont download it"}

final_state = workflow.invoke(initial_state)
final_state

{'review': 'This system is trash, nothing works properly and lags alot. Dont download it',
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'Bug', 'tone': 'angry', 'urgency': 'high'},
 'response': "Hi [User Name],\n\nI understand you're frustrated with the bug you're experiencing and I sincerely apologize for the inconvenience it's causing.  I see you've marked this as high urgency, and I want to assure you that we're taking it seriously.\n\nCould you please provide me with some more details about the bug?  \n\n* What exactly is happening? \n* When did you first notice the issue?\n* What steps have you already taken to try and resolve it?\n\nThe more information you can give me, the faster I can help get this sorted out for you.  \n\nThanks for your patience and understanding. I'm here to help! \n\n\nBest regards,\n\n[Your Name] \nSupport Assistant \n\n"}

In [31]:
print(final_state['response'])

Hi [User Name],

I understand you're frustrated with the bug you're experiencing and I sincerely apologize for the inconvenience it's causing.  I see you've marked this as high urgency, and I want to assure you that we're taking it seriously.

Could you please provide me with some more details about the bug?  

* What exactly is happening? 
* When did you first notice the issue?
* What steps have you already taken to try and resolve it?

The more information you can give me, the faster I can help get this sorted out for you.  

Thanks for your patience and understanding. I'm here to help! 


Best regards,

[Your Name] 
Support Assistant 


